In [4]:
from ShopgunAPI import ShopgunAPI
import json
import io

shopgunAPI = ShopgunAPI(verbose=True);
#shopgunAPI.getStores();

In [5]:
selectedBrandNames = ['coop mega', 'coop extra', 'coop marked', 'kiwi', 'meny', 'rema', 'matkroken',\
                      'coop prix','joker', 'bunnpris', 'extra', 'spar']
selectedBrandNames = ['spar']
selectedCatalogList = shopgunAPI.getCatalogs(selectedBrandNames=selectedBrandNames)

All Catalogs            Store Id    Id
----------------------  ----------  -------
REMA 1000               957crwF     3a2cijz
Extra Sagene            1760ERq     b8c6rkz
Extra Bjølsen           d2bdSIq     172dokz
Obs! Alnabru            3a66jSA     353cU4z
SPAR                    2a7dV81     6782wbz
Coop Prix Marsveien     d496sXz     efe1z4z
Elkjøp                  de03l41     bf67Bnz
Bunnpris Nydalen        f00d501     b3eddiz
Bunnpris Nydalen        f00d501     adb1Lvz
Biltema                 d026SA1     1e3euBz
Biltema                 d026SA1     3957vqE
Jula                    12d0Ta3     462fuUz
Nille                   6981plU     f2adyvz
G-Sport | G-MAX         5a29G48     e883g5z
XXL Majorstuen          6c3a9C3     6aa0W4z
lefdal                  3fc34cy     36709nz
Clas Ohlson             b7e84C1     7432Djz
Rusta                   2d99HSq     15ealnz
Euronics                9dc4G41     42a9Vlz
Apotek 1                d146811     372assz
Jernia                  84d5SND     7

In [6]:
from MatifyAPI import MatifyAPI
matifyAPI = MatifyAPI()
#Get offers in selected catalogs
for catalogStore, catalogId, catalogRunFrom, catalogBrand in selectedCatalogList:
    offerDescList = shopgunAPI.getOfferDescriptionsOfCatalog(catalogId, catalogStore)
    JsonFileName = shopgunAPI.generateCatalogFileName(catalogBrand, catalogId, catalogRunFrom)
    with io.open(JsonFileName, 'w', encoding="utf-8") as json_file:
        data = json.dumps(list(offerDescList), ensure_ascii=False)
        json_file.write(data)
        matifyAPI.uploadCatalog(brandName=catalogBrand, catalogData=data, catalogFileName=JsonFileName)
    print("Done.")

--------------------------------------------
Requesting catalog SPAR with id 6782wbz...
Number of offers: 52
Number of offers after filtered duplicates: 49
Uploading catalog spar_6782wbz_2017-12-03T23:00:00+0000.json
<Response [400]>
{"message": "This file has been uploaded", "status": "Fail"}
Done.


/home/nikasa/miniconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
token = 'b2h6ylyfn6pfvoz5wuvc'
if self.verbose:
    print("Uploading catalog " + catalogFileName)

storeId = self.getStoreId(brandName);
response = self.session.post( SERVER_API_ENDPOINT + 'upload_products/'+str(storeId),
                          data = {"token":token, 
                                  "file_name":catalogFileName, 
                                  "data": catalogData})

## Download deals from before

In [ ]:
selectedBrandNames = ['coop mega', 'coop extra', 'coop marked', 'kiwi', 'meny', 'rema', 'matkroken', 
                      'coop obs', 'coop prix','joker', 'bunnpris', 'extra', 'spar']

catalogsResponse = session.get('https://api.etilbudsavis.dk/v2/catalogs?r_lat=59.95'+
                               '&r_lng=10.75&r_radius=700000&api_av=0.1.33'+
                               '&offset=10&limit=100'+
                               '&run_from=2015-01-30T23:00:00+0000'+
                               '&run_till=2015-02-13T22:59:59+0000',
                               headers=defaultHeaders)
catalogs = json.loads(catalogsResponse.text)
catalogList = [[catalog['branding']['name'], catalog['store_id'], catalog['id']] for catalog in catalogs]
print tabulate(catalogList, headers=['All Catalogs', 'Store Id', 'Id'])

#Check if any brand name contains any selectedBrandName
selectedCatalogs = [catalog for catalog in catalogs 
                    if len([brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower()]) > 0]
selectedCatalogList = [[catalog['branding']['name'], catalog['id'], catalog['run_from'], 
                        next(brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower())] 
                       for catalog in selectedCatalogs]
print "\n---------------------------------"
print tabulate(selectedCatalogList, headers=['Selected Catalogs', 'Id', 'Run From', 'Brand'])

In [ ]:
catalogsResponse

In [ ]:
catalogs = json.loads(catalogsResponse.text)
